Source": 
https://www.analyticsvidhya.com/blog/2020/01/link-prediction-how-to-predict-your-future-connections-on-facebook/

In [1]:
#import all the necessary libraries and modules
import pandas as pd
import numpy as np
import random
import networkx as nx
import matplotlib.pyplot as plt
import csv
import tqdm

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from node2vec import Node2Vec
from sklearn.cluster import KMeans
from node2vec.edges import HadamardEmbedder

In [2]:
# load nodes details
with open("data/train.txt") as f:
    reader = csv.reader(f, delimiter = "\t")
    train_nodes = list(reader)

#create a dict with the source as key as the sinks as values
mydict = {}
for node_line in train_nodes:
    key = node_line.pop(0)
    mydict[key] = node_line

#form our source and target nodes list
node_list1 = []
node_list2 = []
for key,val in mydict.items():
    for i in range(len(val)):
        node_list1.append(key)
        node_list2.append(val[i])
        
#display dataframe
network_df = pd.DataFrame({'source':node_list1, 'sink':node_list2})
print(network_df.shape)
network_df.head()

(24004361, 2)


,source,sink
0,540762,1912140
1,540762,1537559
2,540762,3091331
3,540762,2757277
4,540762,3237295


In [6]:
node_list = list(zip(network_df['source'],network_df['sink']))

In [8]:
G_complete = nx.DiGraph()
G_complete.add_edges_from(node_list)

In [9]:
print(len(G_complete.nodes()))

4867136


In [ ]:
#Randomly sample 0.1% of our entire edge pairs
network_df_sample = network_df.sample(frac = 0.0001).reset_index(drop = True)
network_df_sample.shape

In [ ]:
network_df_sample.head()

node_list1_sampled = list(network_df_sample['source'])
node_list2_sampled = list(network_df_sample['sink'])
node_pairs = list(zip(node_list1_sampled,node_list2_sampled))
print(node_pairs)

In [ ]:
# plot a graph for our sampled pairs
G = nx.from_pandas_edgelist(network_df_sample,source='source',target='sink', edge_attr=None, create_using=nx.DiGraph())

plt.figure(figsize=(20,20))
pos = nx.random_layout(G, seed=23)
nx.draw(G, with_labels=False,  pos = pos, node_size = 40, alpha = 0.6, width = 0.7, connectionstyle='arc3, rad = 0.1')

plt.show()



# combine all nodes in a list
node_list = node_list1_sampled + node_list2_sampled

# remove duplicate items from the list
node_list = list(dict.fromkeys(node_list))

# build adjacency matrix
adj_G = nx.to_numpy_matrix(G, node_list)
adj_G.shape

In [ ]:
# Generate walks
node2vec = Node2Vec(G, dimensions=2, walk_length=20, num_walks=10,workers=4)
# Learn node embeddings 
model = node2vec.fit(window=10, min_count=1)
model.wv.save_word2vec_format("embedding.emb") #save the embedding in file embedding.emb

In [10]:
model.save("word2vec.model") #save the model for later use 
#use model = Word2Vec.load("word2vec.model") to load

In [26]:
#example: find most similar nodes to node 1705425
model.wv.most_similar('3361377')

[('137924', 1.0),
 ('2150273', 0.9999995231628418),
 ('461343', 0.9999979734420776),
 ('4010763', 0.9999911785125732),
 ('4093673', 0.9999781250953674),
 ('1183655', 0.9999607801437378),
 ('1925933', 0.999956488609314),
 ('2934751', 0.9999469518661499),
 ('2036173', 0.999945878982544),
 ('1051108', 0.9999414682388306)]

In [16]:
# Learn link embeddings
edges_embs = HadamardEmbedder(keyed_vectors=model.wv)
edges_kv = edges_embs.as_keyed_vectors()

Generating edge features: 100%|█████████████████████████████████████████| 6249880/6249880.0 [01:43<00:00, 60488.56it/s]


In [20]:
# Save embeddings for later use
edges_kv.save_word2vec_format('edgeembedding.emb')

In [ ]:
#both outputs an array
#model.wv.get_vector('2')
#edges_embs[('1', '2')]

In [21]:
#example: find most similar edges
edges_kv.most_similar(str(('1705425', '3006327')))

KeyError: "word '('1705425', '3006327')' not in vocabulary"

In [ ]:
X = np.loadtxt("embedding.emb", skiprows=1) # load the embedding of the nodes of the graph
print(X)


In [ ]:
# sort the embedding based on node index in the first column in X
X=X[X[:,0].argsort()]; 
print(X)


In [ ]:
Z=X[0:X.shape[0],1:X.shape[1]]; # remove the node index from X and save in Z

kmeans = KMeans(n_clusters=2, random_state=0).fit(Z) # apply kmeans on Z
labels=kmeans.labels_  # get the cluster labels of the nodes.
print(labels)

In [ ]:
print(Z)

# get unconnected node-pairs
all_unconnected_pairs = []

# traverse adjacency matrix
offset = 0
for i in (range(adj_G.shape[0])):
    for j in range(offset,adj_G.shape[1]):
        if i != j: #if not self
            #if nx.shortest_path_length(G, str(i), str(j)) <=2: 
            if adj_G[i,j] == 0:
                  all_unconnected_pairs.append([node_list1_sampled[i],node_list2_sampled[j]])

    offset = offset + 1

#find unconnected pairs
gEdges = G.edges()
unconnected_pairs = set()
for a in G.nodes():
    for b in G.nodes():
        if a != b and (a,b) not in gEdges:
            unconnected_pairs.add( (a, b) )

for pairs in unconnected_pairs:
    pairs = list(pairs)
    
unconnected_pairs = list(unconnected_pairs)

#create negative samples
node_1_unlinked = [i[0] for i in unconnected_pairs]
node_2_unlinked = [i[1] for i in unconnected_pairs]

data = pd.DataFrame({'node_1':node_1_unlinked, 
                     'node_2':node_2_unlinked})

# add target variable 'link'
data['link'] = 0

#Graphs with Graph Convolutional Networks
order = sorted(list(G.nodes()))
A = nx.to_numpy_matrix(G, nodelist=order)

I = np.eye(G.number_of_nodes())
A_hat = A + I
D_hat = np.array(np.sum(A_hat, axis=0))[0]
D_hat = np.matrix(np.diag(D_hat))

W_1 = np.random.normal(
    loc=0, scale=1, size=(G.number_of_nodes(), 4))
W_2 = np.random.normal(
    loc=0, size=(W_1.shape[1], 2))

#Stack the GCN layers. We here use just the identity matrix as feature representation, aka.
#each node is represented as a one-hot encoded categorical variable.
def relu(X):
    return np.maximum(0,X)

def gcn_layer(A_hat, D_hat, X, W):
    return relu(D_hat**-1 * A_hat * X * W)

H_1 = gcn_layer(A_hat, D_hat, I, W_1)
H_2 = gcn_layer(A_hat, D_hat, H_1, W_2)
output = H_2

feature_representations = {int(node): np.array(output)[int(node)] for node in G.nodes()}

feature_representations = {}
ct = 0
for node in order:
    feature_representations[int(node)]=np.array(output)[ct]
    ct +=1

xs,ys = zip(*feature_representations.values())
labels = feature_representations.keys()   

# display
plt.figure(figsize=(10,8))
plt.title('Scatter Plot', fontsize=20)
plt.xlabel('x', fontsize=15)
plt.ylabel('y', fontsize=15)
plt.scatter(xs, ys, marker = 'o')


# Different from above
initial_node_count = len(G.nodes)
network_df_temp = network_df.copy()

# empty list to store removable links
omissible_links_index = []

for i in network_df.index.values:
  
  # remove a node pair and build a new graph
  G_temp = nx.from_pandas_edgelist(network_df_temp.drop(index = i), "source", "sink", create_using=nx.MultiGraph())
  
  # check there is no spliting of graph and number of nodes is same
  if (nx.number_connected_components(G_temp) == 1) and (len(G_temp.nodes) == initial_node_count):
    omissible_links_index.append(i)
    network_df_temp = network_df_temp.drop(index = i)